In [1]:
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/mcmc")

from ase.build import make_supercell
from ase.io import read, write

import catkit
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run
import os
import numpy as np

os.environ["PROJECT_DIR"] = os.getcwd()

In [2]:
# SrTiO3 from HTVS db
import sys
import os
import django
import numpy as np
# from tqdm import tqdm
# import pandas as pd
# import pickle as pkl
sys.path.append('/home/dux/htvs/')
sys.path.append('/home/dux/htvs/djangochem/')

#from imports import *
#from django_imports import *
# from pymatgen.core.periodic_table import Element
# from chemconfigs.vasp.defaults import Magmom
# from ase.visualize import view

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # CAUTION!!! LINE NECESSARY WHEN INTERACTING WITH DATABASE IN JUPYTER NOTEBOOK. DO NOT USE THIS LINE IF YOU ARE PLANNING ON SAVING/REMOVING THINGS FROM THE DATABASE!!!!!

# this must be run to setup access to the django settings and make database access work etc.
django.setup()
# from jobs.models import Job, JobConfig
# from django.contrib.contenttypes.models import ContentType
# from ase.io import write

from pgmols.models import Surface
# from analysis.structure.perovskite import PerovskiteAnalyzer
# from pymatgen.core.periodic_table import Element

unit_slab = Surface.objects.get(id=217485271).as_ase_gratoms()
unit_slab.write('SrTiO3_001_pristine_slab.cif')

# get supercell
# multiplier = np.diag([2, 2, 1])
# slab = make_supercell(unit_slab, multiplier)
slab = unit_slab*(2,2,1)
slab.write('SrTiO3_001_2x2_pristine_slab.cif')

In [3]:
# Pymatgen adsorption sites for InGaN
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

from ase.io import read, write

pristine_slab = slab
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# AdsorbateSiteFinder(slab, selective_dynamics=False, height=0.9, mi_vec=None)
sites = site_finder.find_adsorption_sites(put_inside=False, symm_reduce=False)
# find_adsorption_sites(distance=2.0, put_inside=True, symm_reduce=0.01, near_reduce=0.01, positions=['ontop', 'bridge', 'hollow'], no_obtuse_hollow=True)

In [4]:
for type in sites.keys():
    print(f"{type} sites has total {len(sites[type])}")

ontop sites has total 12
bridge sites has total 36
hollow sites has total 16
all sites has total 64


In [5]:
ads_positions = sites['all']
# print(f"adsorption coordinates are")
# print(ads_positions)
# take just the first 18 sites after inspection
# select_positions = ads_positions[:18]

# take all positions
select_positions = ads_positions

In [6]:
# adsorb onto the sites
from pymatgen.core.structure import Molecule
ads_mol = Molecule(['O'], [[0, 0, 0]])
for ads_coord in select_positions:
    slab = site_finder.add_adsorbate(ads_mol, ads_coord, repeat=None, translate=False, reorient=False)
    site_finder.slab = slab

from pymatgen.io.cif import CifWriter
# write out slab
cif_slab = CifWriter(site_finder.slab)
# cif_slab.write_file("pymatgen_SrTiO3_001_4x4_O_all_adsorbed_slab.cif")
cif_slab.write_file("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")

In [13]:
all_adsorbed = read("pymatgen_SrTiO3_001_2x2_O_all_adsorbed_slab.cif")
print(f"all adsorbed {len(all_adsorbed)}")

select_positions = all_adsorbed.get_positions()[len(pristine_slab):]

all adsorbed 124


In [14]:
select_positions

array([[ 7.87106322,  7.94072982, 19.32035338],
       [ 1.96776173,  1.9512601 , 19.27924584],
       [ 1.9677844 ,  0.14564935, 19.23214617],
       [ 7.87106322,  3.95573464, 19.32035338],
       [ 1.96776173,  5.93625536, 19.27924584],
       [ 1.9677844 ,  4.13064461, 19.23214617],
       [ 3.93552559,  7.94072982, 19.32035338],
       [ 5.90329936,  1.9512601 , 19.27924584],
       [ 5.90332203,  0.14564935, 19.23214617],
       [ 3.93552559,  3.95573464, 19.32035338],
       [ 5.90329936,  5.93625536, 19.27924584],
       [ 5.90332203,  4.13064461, 19.23214617],
       [ 1.96777307,  7.02594761, 19.25569613],
       [ 0.98387488,  6.93849259, 19.29979973],
       [ 5.90331077,  7.02594761, 19.25569613],
       [ 6.88718133,  6.93849259, 19.29979973],
       [ 6.88719266,  0.05819432, 19.27624978],
       [ 7.87106818,  5.93625536, 19.27924584],
       [ 6.88718133,  4.94599496, 19.29979973],
       [ 0.98387488,  4.94599496, 19.29979973],
       [ 2.9516437 ,  6.93849259, 19.299